Chapter 2.a.v.  Equality reductions and conditional assumptions
=======

The final reduction rules involve equality and conditionals.

The **Equals** expression class is defined outside the core in `proveit.logic.equality`.  It does not have any special reduction rule of its own.  However, equality is understood in the core derivation system in a few senses.  Equality is used for requirements when there are a range of lambda parameters and the length of the operands must *equal* the length of the parameter range.  Also, tuples of indices need to be *equal* when performing parameter-depdendent **ExprRange** expansions.  In addition to this, automatic reductions may be effected by *any* expression class via equality.  This can be used to perform any kind of reduction during lambda application (or instantiation, which uses lambda application) with just the requirement that the original sub-expression is equal to the *reduced* sub-expression.  For best practices, however, this should only make obvious reductions in which notation is otherwise awkward and generally unnecessary rather than being used for general simplification.  For example, logical `and`/`or` applied to zero operands or one operand is reduced via $[\land]() = \top$ (`and` applied to zero operands is `TRUE`), $\forall_{A \in \mathbb{B}} [\land](A) = A$ (`and` applied to one operand reduces to just that operand if it is a boolean), $[\lor]() = \bot$ (`or` applied to zero operands is `FALSE`), $\forall_{A \in \mathbb{B}} [\lor](A) = A$ (`or` applied to one operand reduces to just that operand if it is a boolean).  The left side of these equations may emerge in the process of a lambda application (or instantiation) but should be immediately reduced to the right side when possible.  Although it is possible to enable/disable these automatic reductions for each expression class independently, general simplification should be done explicitly (not using this automation) so the user has full and explicit control as they perform they precise derivations they want to attain.

A **Conditional** expression is a core expression type with one special reduction rule.  Specifically, the condition of the **Conditional** may be introduced as an internal assumption in the process of performing a lambda application (or instantiation).  Conditionals are used implicitly in the conditions of quantifiers.  For example, $\forall_{x~|~Q(x)} P(x)$ is internally represented as an operation with the $\forall$ operator applied to the lambda expression $x \mapsto \left\{P(x) \textrm{ if } Q(x)\right..$, where $\left\{P(x) \textrm{ if } Q(x)\right..$ is a **Conditional** expression, defined to be $P(x)$ when $Q(x)$ is `TRUE` but otherwise undefined for any practical purpose.  Since the $P(x)$ value of the **Conditional** is only relevant when $Q(x)$ is `TRUE`, we can freely add $Q(x)$ to the *assumptions* when making replacements of the $P(x)$ expression in the process of performing lambda application (or instantiation).

In [1]:
import proveit
from proveit import defaults
%begin equality_reductions

We will demonstrate both of these reductions simultaneously in the following example.  First, we import a *theorem* which effectively states that we can derive the conjunction of a portion of a tuple if the conjunction of all of the elements of the tuple are known to be true.  *(Note: the condition below uses in implicit conjunction.  We should change the style to make it explicit in this case so it is easier to see what this theorem is doing.)*

In [2]:
from proveit.logic.booleans.conjunction import some_from_and
some_from_and

|- forall_{l, m, n in Natural} [forall_{A_{1}, ..A_{_a}.., A_{l}, B_{1}, ..B_{_a}.., B_{m}, C_{1}, ..C_{_a}.., C_{n} | A_{1}, ..A_{_a}.., A_{l}, B_{1}, ..B_{_a}.., B_{m}, C_{1}, ..C_{_a}.., C_{n}} (B_{1} and ..B_{_a}.. and B_{m})]

If we instantiate $l : 1, m:1, n:1$ (which uses lambda application internally), then are triple of ranges reduces to a triple of singular indexed variable elements via **ExprRange** reduction discussed in a previous chapter.  The $B_1 \land \ldots \land B_m$ conjunction should reduce to $[\land](B_1)$ after the **ExprRange** reduction is performed.  However, an *equality reduction* is additionally performed via $[\land](B_1) = B_1$.

In [3]:
from proveit.numbers import zero, one
from proveit import l, m, n
some_from_and_inst = some_from_and.instantiate({l:one, m:one, n:one})

some_from_and_inst: |- forall_{A_{1}, B_{1}, C_{1} | A_{1}, B_{1}, C_{1}} B_{1}

To satisfy the $[\land](B_1) = B_1$ requirement, we must prove that $B_1 \in \mathbb{B}$.  This is where the special **Conditional** reduction rule is needed.  Because this reduction occurs with the **Conditional** with the *condition* $A_1 \land B_1 \land C_1$, we can utilize this assumption internally (it is not an external assumption and does not appear in the resulting **Judgment**).  Indeed, step 6 below is the requirement $\{ A_1 \land B_1 \land C_1 \} \vdash [\land](B_1) = B_1$, meeting the *equality reduction* requirement by using the assumption form the *condition* of the **Conditional**.  This is done via step 9: $\{ A_1 \land B_1 \land C_1 \} \vdash [\land](B_1) = B_1 \in \mathbb{B}$.

In [4]:
some_from_and_inst.proof()

step type	requirements	statement
0	instantiation	1, 10, 2, 3, 4, 5*	|- forall_{A_{1}, B_{1}, C_{1} | A_{1}, B_{1}, C_{1}} B_{1}
	l : 1, m : 1, n : 1
1	conjecture		|- forall_{l, m, n in Natural} [forall_{A_{1}, ..A_{_a}.., A_{l}, B_{1}, ..B_{_a}.., B_{m}, C_{1}, ..C_{_a}.., C_{n} | A_{1}, ..A_{_a}.., A_{l}, B_{1}, ..B_{_a}.., B_{m}, C_{1}, ..C_{_a}.., C_{n}} (B_{1} and ..B_{_a}.. and B_{m})]
2	instantiation	6	|- (A_{1}, ..A_{a}.., A_{1}) = (A_{1})
	f(_a) : A_{_a}, i : 1
3	instantiation	6	|- (B_{1}, ..B_{a}.., B_{1}) = (B_{1})
	f(_a) : B_{_a}, i : 1
4	instantiation	6	|- (C_{1}, ..C_{a}.., C_{1}) = (C_{1})
	f(_a) : C_{_a}, i : 1
5	instantiation	7, 8	{A_{1} and B_{1} and C_{1}} |- and(B_{1}) = B_{1}
	A : B_{1}
6	conjecture		|- forall_{f, i} ((f(i), ..f(a).., f(i)) = (f(i)))
7	conjecture		|- forall_{A in BOOLEAN} (and(A) = A)
8	instantiation	9, 10, 11, 12, 13	{A_{1} and B_{1} and C_{1}} |- B_{1} in BOOLEAN
	m : 1, n : 1, A : (A_{1}), B : B_{1}, C : (C_{1})
9	conjecture		|- forall_{m, n in Natural} [forall_{A_{1}, ..A_{_a}.., A_{m}, B, C_{1}, ..C_{_a}.., C_{n} | (A_{1} and ..A_{_a}.. and A_{m} and B and C_{1} and ..C_{_a}.. and C_{n}) in BOOLEAN} (B in BOOLEAN)]
10	theorem		|- 1 in Natural
	proveit.numbers.numerals.decimals.nat1
11	instantiation	14	|- |(A_{1})| = |(1, ..k.., 1)|
	a : A_{1}
12	instantiation	14	|- |(C_{1})| = |(1, ..k.., 1)|
	a : C_{1}
13	instantiation	15, 16, 17	{A_{1} and B_{1} and C_{1}} |- (A_{1} and B_{1} and C_{1}) in BOOLEAN
	P(_a) : _a in BOOLEAN, x : A_{1} and B_{1} and C_{1}
14	conjecture		|- forall_{a} (|(a)| = |(1, ..k.., 1)|)
15	theorem		|- forall_{P, x | P(TRUE), x} P(x)
	proveit.logic.equality.substitute_truth
16	conjecture		|- TRUE in BOOLEAN
17	assumption		{A_{1} and B_{1} and C_{1}} |- A_{1} and B_{1} and C_{1}

Note that an asterisk (`*`) marks the requirement of step 6 in the requirements list of step 0.  This asterisk shows up at the bottom of the table with the legend indicating that `*` marks "equality replacement requirements".  The purpose of marking these requirements is to make it simple to verify **Prove-It** proofs independently.  A checker should not need to know what "equality replacements" are available or which ones should be employed for any particular derivation step.  That would not be appropriate since "equality replacements" may be defined outside of the core and may be enabled/disabled at will.  All other derivation step requirements are known according to the core derivation rules which an independent checker will need to understand, but these rules are intended to be finite and stable.  When it comes to the "equality replacements," however, the checker simply needs to note that the marked (`*`-ed) requirements should be used to replace matching occurrences of the left side of the equalities with the respective right side in a straightforward manner.

In the following example, we will instantiation $l:0, m:1, n:0$.  The $A_1, \ldots, A_l$ range and the $C_1, \ldots, C_n$ range will be eliminated via empty range reductions, and $B_1, \ldots, B_1$ reduces to the singular element $B_1$.

In [5]:
some_from_and

|- forall_{l, m, n in Natural} [forall_{A_{1}, ..A_{_a}.., A_{l}, B_{1}, ..B_{_a}.., B_{m}, C_{1}, ..C_{_a}.., C_{n} | A_{1}, ..A_{_a}.., A_{l}, B_{1}, ..B_{_a}.., B_{m}, C_{1}, ..C_{_a}.., C_{n}} (B_{1} and ..B_{_a}.. and B_{m})]

In [6]:
some_from_and.instantiate({l:zero, m:one, n:zero})

|- forall_{B_{1} | B_{1}} B_{1}

Once again, $[\land](B_1)$ within the universal quantification is reduced to $B_1$.  However, note that the $[\land](B_1)$ in the condition is not reduced.  This is because we must know that $B_1$ is a boolean before we can reduce $[\land](B_1) = B_1$.  The $[\land](B_1)$ condition is used to prove $B_1 \in \mathbb{B}$ within the **Conditional** of the universal quantifier, but it cannot be used to justify its own reduction.

Enabling/disabling automatic reductions can be done via the `defaults.auto_reduce` flag and/or the `defaults.disabled_auto_reduction_types` set.  If we add the `And` class to `defaults.disabled_auto_reduction_types`, the reduction demonstrated above will be disabled.

In [7]:
from proveit.logic import And
defaults.disabled_auto_reduction_types.add(And)

In [8]:
some_from_and_inst_no_reduce = some_from_and.instantiate({l:one, m:one, n:one})

some_from_and_inst_no_reduce: |- forall_{A_{1}, B_{1}, C_{1} | A_{1}, B_{1}, C_{1}} and(B_{1})

No reduction of the conjunction is performed.  However, the `Or` reduction can still be performed because this disabled the auto-reduction specifically for `And`.

In [9]:
from proveit.logic.booleans.disjunction import demorgans_law_and_to_or
demorgans_law_and_to_or

|- forall_{m in Natural} [forall_{A_{1}, ..A_{_a}.., A_{m} | not(not(A_{1}) and ..not(A_{i}).. and not(A_{m}))} (A_{1} or ..A_{_a}.. or A_{m})]

In [10]:
demorgans_law_and_to_or.instantiate({m:one})

|- forall_{A_{1} | not(and(not(A_{1})))} A_{1}

Similar to what we saw above when we instantiated `some_from_and` using $l:0, m:1, n:0$, the condition can be used to prove $A_1 \in \mathbb{B}$ within the **Conditional** but cannot be used to justify the reduction of the conditional itself, even if the `And` reduction were not disabled.

Setting `defaults.auto_reduce` to `False` will disable all auto-reductions.

In [11]:
defaults.auto_reduce = False
demorgans_law_and_to_or.instantiate({m:one})

|- forall_{A_{1}, ..A_{_a}.., A_{1} | not(not(A_{1}) and ..not(A_{i}).. and not(A_{1}))} (A_{1} or ..A_{_a}.. or A_{1})

In [12]:
%end equality_reductions

# Next chapter: <a href="tutorial04_relabeling.ipynb">ToDo</a>

## <a href="tutorial00_introduction.ipynb#contents">Table of Contents</a>